# Motivation


## Datasets

## Why did we choose these datasets

## End goal

# Basic Stats

## Choices in data cleaning and preprocessing

## Dataset stats and key points

# Data Analysis

# Genre

## Which genre

## Visual Narrative

## Narrative Structure

# Visualizations

# Website

# Discussion

# Contributions